In [2]:
import torch
import torch.nn as nn
import numpy as np
import torch.nn.functional as F
import torch.optim as optim
import os
import glob
import matplotlib.pyplot as plt
import cv2
import rawpy
# import Dataset calss
from torch.utils.data import Dataset


In [3]:
def pack_raw_sony(raw):
    # pack Bayer image to 4 channels
    im = raw.raw_image_visible.astype(np.float32)
    im = np.maximum(im - 512, 0) / (16383 - 512)  # subtract the black level

    im = np.expand_dims(im, axis=2)
    img_shape = im.shape
    H = img_shape[0]
    W = img_shape[1]

    out = np.concatenate((im[0:H:2, 0:W:2, :],
                          im[0:H:2, 1:W:2, :],
                          im[1:H:2, 1:W:2, :],
                          im[1:H:2, 0:W:2, :]), axis=2)
    return out

In [4]:
# conv1 = slim.conv2d(input, 32, [3, 3], rate=1, activation_fn=lrelu, scope='g_conv1_1')
# conv1 = slim.conv2d(conv1, 32, [3, 3], rate=1, activation_fn=lrelu, scope='g_conv1_2')
# pool1 = slim.max_pool2d(conv1, [2, 2], padding='SAME')

# conv2 = slim.conv2d(pool1, 64, [3, 3], rate=1, activation_fn=lrelu, scope='g_conv2_1')
# conv2 = slim.conv2d(conv2, 64, [3, 3], rate=1, activation_fn=lrelu, scope='g_conv2_2')
# pool2 = slim.max_pool2d(conv2, [2, 2], padding='SAME')

# conv3 = slim.conv2d(pool2, 128, [3, 3], rate=1, activation_fn=lrelu, scope='g_conv3_1')
# conv3 = slim.conv2d(conv3, 128, [3, 3], rate=1, activation_fn=lrelu, scope='g_conv3_2')
# pool3 = slim.max_pool2d(conv3, [2, 2], padding='SAME')

# conv4 = slim.conv2d(pool3, 256, [3, 3], rate=1, activation_fn=lrelu, scope='g_conv4_1')
# conv4 = slim.conv2d(conv4, 256, [3, 3], rate=1, activation_fn=lrelu, scope='g_conv4_2')
# pool4 = slim.max_pool2d(conv4, [2, 2], padding='SAME')

# conv5 = slim.conv2d(pool4, 512, [3, 3], rate=1, activation_fn=lrelu, scope='g_conv5_1')
# conv5 = slim.conv2d(conv5, 512, [3, 3], rate=1, activation_fn=lrelu, scope='g_conv5_2')

# up6 = upsample_and_concat(conv5, conv4, 256, 512)
# conv6 = slim.conv2d(up6, 256, [3, 3], rate=1, activation_fn=lrelu, scope='g_conv6_1')
# conv6 = slim.conv2d(conv6, 256, [3, 3], rate=1, activation_fn=lrelu, scope='g_conv6_2')

# up7 = upsample_and_concat(conv6, conv3, 128, 256)
# conv7 = slim.conv2d(up7, 128, [3, 3], rate=1, activation_fn=lrelu, scope='g_conv7_1')
# conv7 = slim.conv2d(conv7, 128, [3, 3], rate=1, activation_fn=lrelu, scope='g_conv7_2')

# up8 = upsample_and_concat(conv7, conv2, 64, 128)
# conv8 = slim.conv2d(up8, 64, [3, 3], rate=1, activation_fn=lrelu, scope='g_conv8_1')
# conv8 = slim.conv2d(conv8, 64, [3, 3], rate=1, activation_fn=lrelu, scope='g_conv8_2')

# up9 = upsample_and_concat(conv8, conv1, 32, 64)
# conv9 = slim.conv2d(up9, 32, [3, 3], rate=1, activation_fn=lrelu, scope='g_conv9_1')
# conv9 = slim.conv2d(conv9, 32, [3, 3], rate=1, activation_fn=lrelu, scope='g_conv9_2')

# conv10 = slim.conv2d(conv9, 27, [1, 1], rate=1, activation_fn=None, scope='g_conv10')
# out = tf.depth_to_space(conv10, 3)

# def upsample_and_concat(x1, x2, output_channels, in_channels):
#     pool_size = 2
#     deconv_filter = nn.Parameter(torch.randn(output_channels, in_channels, pool_size, pool_size))
#     deconv = nn.functional.conv_transpose2d(x1, deconv_filter, stride=pool_size)

#     # Concatenate along the channel dimension
#     deconv_output = torch.cat([deconv, x2], dim=1)

#     return deconv_output

# class UNet(nn.Module):
#     def __init__(self):
#         super().__init__()
#         self.conv1 = nn.conv2d(in_channels=512, out_channels=32, kernel_size=3, stride=1, padding=0, dilation=1)
#         self.conv2 = nn.conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=0, dilation=1)
#         self.conv3 = nn.conv2d(in_channels=64, out_channels=128, kernel_size=3, stride=1, padding=0, dilation=1)
#         self.conv4 = nn.conv2d(in_channels=128, out_channels=256, kernel_size=3, stride=1, padding=0, dilation=1)
#         self.conv5 = nn.conv2d(in_channels=256, out_channels=512, kernel_size=3, stride=1, padding=0, dilation=1)
#         self.conv6 = nn.conv2d(in_channels=512, out_channels=256, kernel_size=3, stride=1, padding=0, dilation=1)
#         self.conv7 = nn.conv2d(in_channels=256, out_channels=128, kernel_size=3, stride=1, padding=0, dilation=1)
#         self.conv8 = nn.conv2d(in_channels=128, out_channels=64, kernel_size=3, stride=1, padding=0, dilation=1)
#         self.conv9 = nn.conv2d(in_channels=64, out_channels=32, kernel_size=3, stride=1, padding=0, dilation=1)
#         self.conv10 = nn.conv2d(in_channels=32, out_channels=12, kernel_size=1, stride=1, padding=0, dilation=1)

#         self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1)
#         self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1)
#         self.pool3 = nn.MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1)
#         self.pool4 = nn.MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1)

#         self.up6 = nn.ConvTranspose2d(512, 256, 2, stride=2)
#         self.up7 = nn.ConvTranspose2d(256, 128, 2, stride=2)
#         self.up8 = nn.ConvTranspose2d(128, 64, 2, stride=2)
#         self.up9 = nn.ConvTranspose2d(64, 32, 2, stride=2)
        
#         self.lrelu = nn.LeakyReLU(0.1)

    
#     def forward(self, x):
#         x = self.conv1.forward(x)
#         x = self.lrelu(x)
#         x = self.pool1.forward(x)

#         x = self.conv2.forward(x)
#         x = self.lrelu(x)
#         x = self.pool2.forward(x)

#         x = self.conv3.forward(x)
#         x = self.lrelu(x)
#         x = self.pool3.forward(x)

#         x = self.conv4.forward(x)
#         x = self.lrelu(x)
#         x = self.pool4.forward(x)

#         x = self.conv5.forward(x)
#         x = self.lrelu(x)
#         x = self.pool5.forward(x)

#         x = self.up6.forward(x)
#         x = self.conv6.forward(x)
#         x = self.lrelu(x)

#         x = self.up7.forward(x)
#         x = self.conv7.forward(x)
#         x = self.lrelu(x)

#         x = self.up8.forward(x)
#         x = self.conv8.forward(x)
#         x = self.lrelu(x)

#         x = self.up9.forward(x)
#         x = self.conv9.forward(x)
#         x = self.lrelu(x)

#         x = self.conv10.forward(x)
#         x = self.lrelu(x)
        
#         output = nn.functional.pixel_shuffle(x, 10)

#         return output

In [4]:
import torch
import torch.nn as nn

# From: https://github.com/nikhilroxtomar/Semantic-Segmentation-Architecture/blob/main/PyTorch/unet.py

""" Convolutional block:
    It follows a two 3x3 convolutional layer, each followed by a batch normalization and a relu activation.
"""
class conv_block(nn.Module):
    def __init__(self, in_c, out_c):
        super().__init__()

        self.conv1 = nn.Conv2d(in_c, out_c, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(out_c)

        self.conv2 = nn.Conv2d(out_c, out_c, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(out_c)

        self.relu = nn.ReLU()

    def forward(self, inputs):
        x = self.conv1(inputs)
        x = self.bn1(x)
        x = self.relu(x)

        x = self.conv2(x)
        x = self.bn2(x)
        x = self.relu(x)

        return x

""" Encoder block:
    It consists of an conv_block followed by a max pooling.
    Here the number of filters doubles and the height and width half after every block.
"""
class encoder_block(nn.Module):
    def __init__(self, in_c, out_c):
        super().__init__()

        self.conv = conv_block(in_c, out_c)
        self.pool = nn.MaxPool2d((2, 2))

    def forward(self, inputs):
        x = self.conv(inputs)
        p = self.pool(x)

        return x, p

""" Decoder block:
    The decoder block begins with a transpose convolution, followed by a concatenation with the skip
    connection from the encoder block. Next comes the conv_block.
    Here the number filters decreases by half and the height and width doubles.
"""
class decoder_block(nn.Module):
    def __init__(self, in_c, out_c):
        super().__init__()

        self.up = nn.ConvTranspose2d(in_c, out_c, kernel_size=2, stride=2, padding=0)
        self.conv = conv_block(out_c+out_c, out_c)

    def forward(self, inputs, skip):
        x = self.up(inputs)
        x = torch.cat([x, skip], axis=1)
        x = self.conv(x)

        return x


class build_unet(nn.Module):
    def __init__(self):
        super().__init__()

        """ Encoder """
        self.e1 = encoder_block(3, 64)
        self.e2 = encoder_block(64, 128)
        self.e3 = encoder_block(128, 256)
        self.e4 = encoder_block(256, 512)

        """ Bottleneck """
        self.b = conv_block(512, 1024)

        """ Decoder """
        self.d1 = decoder_block(1024, 512)
        self.d2 = decoder_block(512, 256)
        self.d3 = decoder_block(256, 128)
        self.d4 = decoder_block(128, 64)

        """ Classifier """
        self.outputs = nn.Conv2d(64, 1, kernel_size=1, padding=0)

    def forward(self, inputs):
        """ Encoder """
        s1, p1 = self.e1(inputs)
        s2, p2 = self.e2(p1)
        s3, p3 = self.e3(p2)
        s4, p4 = self.e4(p3)

        """ Bottleneck """
        b = self.b(p4)

        """ Decoder """
        d1 = self.d1(b, s4)
        d2 = self.d2(d1, s3)
        d3 = self.d3(d2, s2)
        d4 = self.d4(d3, s1)

        """ Classifier """
        outputs = self.outputs(d4)

        return outputs

In [6]:
input_dir = '/home/deeplearning/images/SonyImages/Sony/short/'
gt_dir = '/home/deeplearning/images/SonyImages/Sony/long/'

# input_dir = 'images/SonyImages/Sony/short/'
# gt_dir = 'images/SonyImages/Sony/long/'

ps = 512  # patch size for training
save_freq = 500

DEBUG = 0
if DEBUG == 1:
    save_freq = 2
    train_ids = train_ids[0:5]

def pack_raw(raw):
    # pack Bayer image to 4 channels
    im = raw.raw_image_visible.astype(np.float32)
    im = np.maximum(im - 512, 0) / (16383 - 512)  # subtract the black level

    im = np.expand_dims(im, axis=2)
    img_shape = im.shape
    H = img_shape[0]
    W = img_shape[1]

    out = np.concatenate((im[0:H:2, 0:W:2, :],
                          im[0:H:2, 1:W:2, :],
                          im[1:H:2, 1:W:2, :],
                          im[1:H:2, 0:W:2, :]), axis=2)
    return out

def random_crop(input, gt, patch_size):
    H = input.shape[0]
    W = input.shape[1]

    xx = np.random.randint(0, W - patch_size)
    yy = np.random.randint(0, H - patch_size)

    input = input[yy:yy + patch_size, xx:xx + patch_size, :]
    gt = gt[yy * 2:yy * 2 + patch_size * 2, xx * 2:xx * 2 + patch_size * 2, :]

    return input, gt

def get_patch(input, gt, patch_size, ix=-1, iy=-1):
    if ix == -1:
        ix = np.random.randint(0, input.shape[1] - patch_size)
    if iy == -1:
        iy = np.random.randint(0, input.shape[0] - patch_size)
    input = input[iy:iy + patch_size, ix:ix + patch_size, :]
    gt = gt[iy * 2:iy * 2 + patch_size * 2, ix * 2:ix * 2 + patch_size * 2, :]

    return input, gt

def input_transform(*arg):
    ret = []
    for a in arg:
        tmp = np.minimum(a / 16383, 1)
        ret.append(tmp)
    return ret

def gt_transform(*arg):
    ret = []
    for a in arg:
        tmp = np.minimum(a / 16383, 1)
        ret.append(tmp)
    return ret

def get_training_data(batch_size=1):
    train_fns = glob.glob(input_dir + '0*.ARW')
    train_ids = [int(os.path.basename(train_fn)[0:5]) for train_fn in train_fns]

    # get random sampple
    train_ids = np.random.choice(train_ids, batch_size)


    inputs = [None] * batch_size
    gts = [None] * batch_size
    for i in range(batch_size):
        train_id = train_ids[i]
        in_files = glob.glob(input_dir + '%05d_00*.ARW' % train_id)
        in_path = in_files[np.random.random_integers(0, len(in_files) - 1)]
        in_fn = os.path.basename(in_path)

        gt_files = glob.glob(gt_dir + '%05d_00*.ARW' % train_id)
        gt_path = gt_files[0]
        gt_fn = os.path.basename(gt_path)
        gt_exposure = float(gt_fn[9:-5])
        in_exposure = float(in_fn[9:-5])
        ratio = min(gt_exposure / in_exposure, 300)

        raw = rawpy.imread(in_path)
        input_full = np.expand_dims(pack_raw(raw), axis=0) * ratio

        raw = rawpy.imread(gt_path)
        gt_full = np.expand_dims(pack_raw(raw), axis=0)

        H = input_full.shape[1]
        W = input_full.shape[2]

        xx = np.random.randint(0, W - ps)
        yy = np.random.randint(0, H - ps)

        input = input_full[:, yy:yy + ps, xx:xx + ps, :]
        gt = gt_full[:, yy * 2:yy * 2 + ps * 2, xx * 2:xx * 2 + ps * 2, :]

        input = np.minimum(input, 1.0)

        inputs[i] = np.transpose(input, (0, 3, 1, 2))
        gts[i] = np.transpose(gt, (0, 3, 1, 2))

    return np.concatenate(inputs, 0), np.concatenate(gts, 0)

def get_test_data(input_path, gt_path, in_exposure, gt_exposure):
    in_files = glob.glob(input_path + '0*.ARW')
    in_path = in_files[0]
    in_fn = os.path.basename(in_path)

    gt_files = glob.glob(gt_path + '0*.ARW')
    gt_path = gt_files[0]
    gt_fn = os.path.basename(gt_path)

    ratio = min(gt_exposure / in_exposure, 300)

    raw = rawpy.imread(in_path)
    input_full = np.expand_dims(pack_raw(raw), axis=0) * ratio

    raw = rawpy.imread(gt_path)
    gt_full = np.expand_dims(pack_raw(raw), axis=0)

    input_full = np.minimum(input_full, 1.0)

    return np.transpose(input_full, (0, 3, 1, 2)), np.transpose(gt_full, (0, 3, 1, 2))

Net = build_unet()

# set up network optimizer, grad etc
optimizer = torch.optim.Adam(Net.parameters(), lr=1e-4)
mse_loss = torch.nn.MSELoss(size_average=False)

# load data
input_full, gt_full = get_training_data(1)
input_full = torch.from_numpy(input_full)
gt_full = torch.from_numpy(gt_full)

# train
for epoch in range(5):
    # begin training
    Net.train()
    optimizer.zero_grad()
    input = (input_full)
    gt = (gt_full)

    print('input size = %s' % str(input.size()))
    
    out = Net(input)

    loss = mse_loss(out, gt)
    loss.backward()
    optimizer.step()


    # Net.train()
    # optimizer.zero_grad()
    # input = Variable(input_full)
    # gt = Variable(gt_full)

    # print('input size = %s' % str(input.size()))

    # out = Net(input)

    # loss = mse_loss(out, gt)
    # loss.backward()
    # optimizer.step()

    # print('Epoch %d, loss = %f' % (epoch, loss.data[0]))

C:\Users\Rico\.conda\envs\assignment3_cuda\lib\site-packages\torch\nn\_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))
C:\Users\Rico\AppData\Local\Temp\ipykernel_17952\2350612420.py:80: DeprecationWarning: This function is deprecated. Please call randint(0, 1 + 1) instead
  in_path = in_files[np.random.random_integers(0, len(in_files) - 1)]


input size = torch.Size([1, 4, 512, 512])


RuntimeError: Given groups=1, weight of size [64, 3, 3, 3], expected input[1, 4, 512, 512] to have 3 channels, but got 4 channels instead